<a href="https://colab.research.google.com/github/rajabhupati/AI-Session/blob/main/RAG_with_HuggingFace_and_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG with Hugging Face Models on Colab
This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) setup using:
- `mistralai/Mistral-7B-Instruct-v0.1`
- `FAISS` for semantic search
- Custom documents loaded and queried

*Note: This is a simplified demo, suitable for small-scale retrieval tasks in Colab.*

In [ ]:
# Install required packages
!pip install -q transformers sentence-transformers faiss-cpu accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [ ]:
import warnings
# Suppress specific UserWarnings related to computation
warnings.filterwarnings("ignore", message="Input type into Linear4bit")

In [ ]:
import logging

# Suppress info-level messages in transformers
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
# Load the embedding model
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Sample documents to index
docs = [
    "Machine learning is a field of AI focused on training systems to learn from data.",
    "RAG stands for Retrieval-Augmented Generation and improves factual accuracy.",
    "Transformers have revolutionized natural language processing.",
    "FAISS is a library for efficient similarity search."
]

doc_embeddings = embed_model.encode(docs, convert_to_numpy=True)

# Create FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

In [ ]:
# Define a search function
def retrieve(query, top_k=1):
    query_vector = embed_model.encode([query])
    D, I = index.search(np.array(query_vector), top_k)
    return [docs[i] for i in I[0]]

## Load the language model (Mistral)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from google.colab import userdata

# Directly set the Hugging Face token in your code (not ideal for real use due to exposure)
huggingface_token = userdata.get('HUGGINGFACE_TOKEN')


model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Ensure token is accessible
if not huggingface_token:
    raise ValueError("HUGGINGFACE_TOKEN is not set. Please ensure it's configured correctly.")

tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    token=huggingface_token
)

def generate(prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Example RAG Flow

In [ ]:
user_query = "What is RAG in AI?"
retrieved = retrieve(user_query)[0]

rag_prompt = f"Use the following context to answer the question:\nContext: {retrieved}\n\nQuestion: {user_query}\nAnswer:"
print(generate(rag_prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following context to answer the question:
Context: RAG stands for Retrieval-Augmented Generation and improves factual accuracy.

Question: What is RAG in AI?
Answer: RAG is an AI technique that stands for Retrieval-Augmented Generation and improves factual accuracy.
